# 8.4.5 선형회귀 분석 실습

##### 보스턴 집값 데이터 셋을 이용하여 집값을 예측하는 모형을 만들어본다.

##### **릿지 회귀 분석**, **라쏘 회귀 분석**, **엘라스틱 넷**을 사용한 결과도 비교한다.
---



# Dataset import
## 데이터 불러오기

In [2]:
import pandas as pd
import numpy as np

In [5]:
# pandas, numpy 불러오지 않고 쓰면 긴 메세지가 뜨는 것을 확인할 수 있었다.

from sklearn import datasets
raw_boston = datasets.load_boston()

In [6]:
# 데이터 셋 내 피처 살펴보기
raw_boston.feature_names

array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT'], dtype='<U7')

# 피처, 타깃 데이터 지정

In [9]:
X = raw_boston.data
y = raw_boston.target

## 트레이닝, 테스트 데이터 분할

In [13]:
from sklearn.model_selection import train_test_split
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state = 1)

## 데이터 표준화

In [15]:
from sklearn.preprocessing import StandardScaler
std_scale = StandardScaler()
std_scale.fit(X_tn)                   # X 트레이닝 데이터 기준으로 std scaler fitting
X_tn_std = std_scale.transform(X_tn)
X_te_std = std_scale.transform(X_te)

# 선형 회귀 분석 모델

먼저 선형 회귀 모델을 만들어 예측해보도록 한다.

## 데이터 학습

In [16]:
from sklearn.linear_model import LinearRegression
clf_lr = LinearRegression()    # 사용할 모델 이름 classifier _ linear regression
clf_lr.fit(X_tn_std, y_tn)     # 피팅

LinearRegression()

## 선형 회귀 모델의 계수, 상수항 확인

선형회귀는 $ y = wx +b $ 형태로 볼 수 있었다!

모형 적합 후 coef_ method를 활용하여 추정된 회귀 계수(w)를 확인해 볼 수 있다.

또한 intercept_ method를 활용하여 추정된 상수항($b$)을 확인해 볼 수 있다.

In [17]:
# 회귀 계수 확인

print(clf_lr.coef_)

[-1.07145146  1.34036243  0.26298069  0.66554537 -2.49842551  1.97524314
  0.19516605 -3.14274974  2.66736136 -1.80685572 -2.13034748  0.56172933
 -4.03223518]


In [18]:
# 상수항

print(clf_lr.intercept_)

22.344591029023768


# L2제약식을 적용한 릿지 회귀 분석 모델

기본 선형 회귀 분석에 L2 제약식을 적용한 **릿지 회귀 분석**을 실습해보자.

## 데이터 학습

In [19]:
from sklearn.linear_model import Ridge    # Ridge Regression
clf_ridge = Ridge(alpha = 1)              # 기본은 1, 반드시 양수여야하고 값이 클 수록 강한 제약식.
clf_ridge.fit(X_tn_std, y_tn)

Ridge(alpha=1)

## 릿지 회귀 모델의 계수, 상수항 확인

In [20]:
# 회귀 계수 확인

print(clf_ridge.coef_)

[-1.05933451  1.31050717  0.23022789  0.66955241 -2.45607567  1.99086611
  0.18119169 -3.09919804  2.56480813 -1.71116799 -2.12002592  0.56264409
 -4.00942448]


In [21]:
# 상수항

print(clf_ridge.intercept_)

22.344591029023768


신기하게도 거의 비슷한따 상수항은 인간 입장에서는 일치한다고 봐도 무방할 정도이다.

# L1제약식을 적용한 라쏘 회귀 분석 모델

이번엔 기본 선형 회귀 분석에 L1제약식을 적용한 **라쏘 회귀 분석**을 실습해보자.

## 데이터 학습

In [22]:
from sklearn.linear_model import Lasso    # Lasso Regression
clf_lasso = Lasso(alpha = 0.01)           # 기본은 1, 릿지와 마찬가지로 제약의 정도를 나타냄
clf_lasso.fit(X_tn_std, y_tn)

Lasso(alpha=0.01)

## 라쏘 회귀 모델의 계수, 상수항 확인

In [23]:
# 계수 확인

print(clf_lasso.coef_)

[-1.04326518  1.27752711  0.1674367   0.66758228 -2.41559964  1.99244179
  0.14733958 -3.09473711  2.46431135 -1.60552274 -2.11046422  0.55200229
 -4.00809905]


In [25]:
# 상수항 확인

print(clf_lasso.intercept_)

22.344591029023768


계수는 조금씩 차이가 나지만 상수항은 같게 나왔다.

# ElasticNet 모델

이번에는 엘라스틱 넷을 사용해보도록 한다.

## 데이터 학습

In [30]:
from sklearn.linear_model import ElasticNet
clf_elastic = ElasticNet(alpha = 0.01, l1_ratio = 0.01) # alpha는 L1제약식의 크기와 L2제약식의 크기를 합한 전체 제약식의 크기
                                                        # l1_ratio는 전체 제약의 크기인 alpha 값에서 L1제약이 차지하는 비율을 의미
                                                        # l1_ratio=0이면 L1제약없이 L2제약만 적용되는 릿지 회귀분석을 의미
clf_elastic.fit(X_tn_std, y_tn)

ElasticNet(alpha=0.01, l1_ratio=0.01)

## 엘라스틱넷 계수, 상수항 확인

In [31]:
# 계수 확인

print(clf_elastic.coef_)

[-1.02916603  1.23681955  0.15236504  0.67859622 -2.34646781  2.02965524
  0.14575132 -2.98592423  2.32013379 -1.48829485 -2.09271972  0.56506801
 -3.9495281 ]


In [32]:
# 상수항 확인

print(clf_elastic.intercept_)

22.344591029023768


# 데이터 예측
만들어본 기본 선형회귀, 릿지 회귀, 라쏘 회귀, 엘라스틱넷 회귀 모델로 데이터 predict를 진행해본다.

## 데이터 예측

In [34]:
pred_lr = clf_lr.predict(X_te_std)
pred_ridge = clf_ridge.predict(X_te_std)
pred_lasso = clf_lasso.predict(X_te_std)
pred_elastic = clf_elastic.predict(X_te_std)

## 모형평가 :  R제곱값

R제곱값으로 각 모형들을 평가해본다. 0과 1사이값을 가지며, 높을수록 좋은 성능을 의미한다.

In [36]:
# import library
from sklearn.metrics import r2_score as r2

# each model test
print("Linear regression: ", r2(y_te, pred_lr))
print("Ridge regression: ", r2(y_te, pred_ridge))
print("Lasso regression: ", r2(y_te, pred_lasso))
print("ElasticNet regression: ", r2(y_te, pred_elastic))

Linear regression:  0.7789410172622859
Ridge regression:  0.7789704562726603
Lasso regression:  0.7787621490259895
ElasticNet regression:  0.7787876079239252


R제곱값에서는 근소한 차이로 릿지 회귀 모형이 좋은 성능을 보인 것으로 나왔으나, 정말 큰 차이가 나지 않는다.

## 모형평가 : MSE(mean squared error! 오차 제곱 합의 평균)

이번에는 mse로 평가해보도록 한다. **'오차'**의 제곱의 합의 평균이므로 작을 수록 좋은 성능을 의미한다.